## 回答のヤバさを図る指標
+0.006と自分たちが作った特徴量の中では圧倒的に光り輝いていた😇

### 作り方

train全部を使って各問題の各選択肢がどれくらい選択されているかを計算

例: content_id=XXXX の選択肢1/2/3/4の選択率: 9% / 5% / 1% / 85%

各問題で選択率を積み上げて各選択肢のパーセンタイルを算出

例:
- 選択肢1: 15% (=1+5+9)
- 選択肢2: 6% (=1+5)
- 選択肢3: 1%
- 選択肢4: 100% (=1+5+9+85)

各ユーザーの過去の選択肢のパーセンタイルをAggregation(std, avg, min, etc.)


### 気持ち
- ほとんどの人が選んでいないようなヤバイ選択肢を選んでる人は多分ヤバイ
- よくできる人はたとえ間違ったとしてもヤバイ選択肢は選ばないはず
- ↑の例の選択肢3とか選ぶ人は多分ヤバイのでそのあともヤバイはず
- 多分この考えは合っててstdの集計がめちゃくちゃ効いていた

In [1]:
import cudf
import pandas as pd
print('cudf_version: ', cudf.__version__)
print('pd_version: ', pd.__version__)

cudf_version:  21.10.01
pd_version:  1.3.5


In [2]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb

In [2]:
%%time
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'

# read data
#feld_needed = ['row_id','timestamp', 'user_id', 'content_id', 'content_type_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation']
#train = pd.read_pickle(train_pickle)[feld_needed]
train = pd.read_pickle(train_pickle)
target = pd.Series(train['user_id'].unique()).sample(frac=0.1)
train = train[train['user_id'].isin(target)]

CPU times: user 2.46 s, sys: 3.15 s, total: 5.61 s
Wall time: 5.63 s


In [3]:
train.query('user_answer != -1').groupby('content_id')['user_answer'].value_counts(normalize=True)

content_id  user_answer
0           0              0.900901
            1              0.040541
            2              0.039039
            3              0.019520
1           1              0.875519
                             ...   
13521       1              0.021978
13522       3              0.896552
            1              0.045977
            2              0.045977
            0              0.011494
Name: user_answer, Length: 49718, dtype: float64

In [4]:
print(train.shape)

(9843503, 13)


In [22]:
train_user_answer = train.query('user_answer != -1').groupby('content_id')['user_answer']\
                         .value_counts(normalize=True).rename('user_answer_rate').reset_index()

In [23]:
pd.merge(train, train_user_answer, on=['content_id', 'user_answer'], how='left')

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,max_time_stamp,rand_time_stamp,viretual_time_stamp,user_answer_rate
0,94633810,0,2008685425,5987,0,0,2,0,NaN,<NA>,1345060,3572034,3572034,0.208359
1,94633811,32454,2008685425,727,0,1,0,1,22000.0,False,1345060,3572034,3604488,0.835763
2,59113467,0,1254081030,4256,0,0,1,0,NaN,<NA>,2513583287,3853868,3853868,0.260251
3,59113468,17948,1254081030,4811,0,1,0,1,29000.0,False,2513583287,3853868,3871816,0.450047
4,94633812,303593,2008685425,376,0,2,1,1,29000.0,False,1345060,3572034,3875627,0.683835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9843498,90322535,1317698272,1918209349,1373,0,348,3,1,19000.0,True,2354172496,80938490647,82256188919,0.633362
9843499,58693447,3889773978,1245758583,255,0,958,0,0,17000.0,True,4574316625,78366420277,82256194255,0.189927
9843500,41038232,35532,871981876,7876,0,1,2,0,29000.0,False,1082172,82256170308,82256205840,0.386992
9843501,23950095,1557694939,511666221,6155,0,172,3,1,19000.0,True,5616951744,80698525567,82256220506,0.544171


In [146]:
"""
train_user_answer_pivot = train_user_answer.reset_index(name='counts')\
                        .pivot(columns='user_answer', values='counts', index='content_id')

train_user_answer_pivot.columns = ['user_answer_' + str(col) for col in train_user_answer_pivot.columns.values]

train_user_answer_pivot
"""

"\ntrain_user_answer_pivot = train_user_answer.reset_index(name='counts')                        .pivot(columns='user_answer', values='counts', index='content_id')\n\ntrain_user_answer_pivot.columns = [col[0] + '_' + col[1] for col in train_user_answer_pivot.columns.values]\n\ntrain_user_answer_pivot\n"

## cudf の場合

In [3]:
%%time

data = cudf.read_csv("../input/riiid-test-answer-prediction/train.csv")
print("Train size:", data.shape)

Train size: (101230332, 10)
CPU times: user 2.12 s, sys: 606 ms, total: 2.72 s
Wall time: 2.73 s


In [4]:
data.head(10)

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,<NA>,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False
5,5,157063,115,156,0,5,2,1,5000.0,False
6,6,176092,115,51,0,6,0,1,17000.0,False
7,7,194190,115,50,0,7,3,1,17000.0,False
8,8,212463,115,7896,0,8,2,1,16000.0,False
9,9,230983,115,7863,0,9,0,1,16000.0,False


In [4]:
data_pdf = data.query('user_answer != -1').to_pandas().groupby('content_id')['user_answer'].value_counts(normalize=True)

In [5]:
data_gdf = cudf.Series.from_pandas(data_pdf)
data_gdf

content_id  user_answer
0           0              0.907721
            1              0.049544
            2              0.030277
            3              0.012458
1           1              0.890646
                             ...   
13521       3              0.011494
13522       3              0.913953
            1              0.046512
            2              0.031395
            0              0.008140
Name: user_answer, Length: 52072, dtype: float64

In [6]:
gdf_user_answer = data_gdf.rename('user_answer_rate').reset_index()
gdf_user_answer

,content_id,user_answer,user_answer_rate
0,0,0,0.907721
1,0,1,0.049544
2,0,2,0.030277
3,0,3,0.012458
4,1,1,0.890646
...,...,...,...
52067,13521,3,0.011494
52068,13522,3,0.913953
52069,13522,1,0.046512
52070,13522,2,0.031395


In [7]:
data_merge = cudf.merge(data, gdf_user_answer, on=['content_id','user_answer'], how='left')
data_merge

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_answer_rate
0,14688,1339558109,141455,6616,0,266,1,1,40750.0,True,0.736569
1,14689,1339558109,141455,6614,0,266,3,1,40750.0,True,0.196568
2,14690,1339558109,141455,6613,0,266,2,0,40750.0,True,0.088882
3,14691,1339558109,141455,6615,0,266,0,1,40750.0,True,0.776508
4,14692,1339781874,141455,6498,0,267,3,1,36500.0,True,0.870648
...,...,...,...,...,...,...,...,...,...,...,...
101230327,101221851,1206491673,2147261668,4516,0,33,2,0,10000.0,True,0.065968
101230328,101221852,1206530707,2147261668,8503,0,34,3,0,33000.0,True,0.127028
101230329,101221853,1206562320,2147261668,8398,0,35,2,1,26000.0,True,0.334573
101230330,101221854,1288523696,2147261668,4821,0,36,3,0,20000.0,True,0.390813


In [9]:
gdf_user_answer.to_csv('../input/user_answer_rate.csv', index=False)

In [10]:
data_merge.to_pandas().to_csv('../input/all_train.csv', index=False)